task name: Create SymbolicSgdLogisticRegression Notebook

task description: Create a dotnet interactive notebook for the provided C# code

In [ ]:
#r "nuget:Microsoft.ML.FastTree"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50)
                .Select(x => x ? randomFloat() : randomFloat() + 0.1f).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000).ToList();
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline = mlContext.BinaryClassification.Trainers.SgdCalibrated();
var model = pipeline.Fit(trainingData);

In [ ]:
private class Prediction
{
    // Original label.
    public bool Label { get; set; }
    // Predicted label from the trainer.
    public bool PredictedLabel { get; set; }
}

var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);
var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();

In [ ]:
foreach (var p in predictions.Take(5))
{
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");
}

In [ ]:
var metrics = mlContext.BinaryClassification.Evaluate(transformedTestData);